In [15]:
# General
import numpy as np
from numpy import inf
import pandas as pd
import os

# Computing
import scipy.io
import sklearn 

# Neuroimaging
import nibabel as nib
from nilearn.datasets import fetch_atlas_schaefer_2018
import hcp_utils as hcp
from nilearn.connectome import ConnectivityMeasure

In [2]:
codedir = os.path.abspath('')  # obtain current direction from which script is runnning

datadir_day2 = '/data/p_02378/data/hcp_rsfunc/hcp-functional-connectivity/'

datadir_day1 = '/data/p_02378/data/hcp_rsfunc/HCP_S1200_rfMRI_DAY1/'

dataout = '/data/p_02667/sex_diff_gradients/data/fc_matrices/schaefer_output/'

In [3]:
path_list = os.listdir(datadir_day2)
path_list.sort()

In [4]:
# Schaefer_400 - 7 networks
dlabel_400 = nib.load('/data/p_02667/sex_diff_gradients/data/schaefer400/Schaefer2018_400Parcels_7Networks_order.dlabel.nii').get_fdata()

# fs_LR_32k is a standard brain surface (like MNI space) -> LR means the first 32k belong to left hemisphere, the second belong to right hemisphere
mat = scipy.io.loadmat('/data/p_02667/sex_diff_gradients/data/schaefer400/fs_LR_32k_medial_mask.mat')


# HCP cortex data does not contain medial_wall using python, we need to delete it in Schaefer template
mask = mat['medial_mask']
schaefer_400 = dlabel_400.T[np.where(mask==1)]


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [5]:
path_list

['.datalad',
 '.git',
 '.gitattributes',
 '.gitmodules',
 '.hcp',
 '100206',
 '100307',
 '100408',
 '100610',
 '101006',
 '101107',
 '101309',
 '101410',
 '101915',
 '102008',
 '102109',
 '102311',
 '102513',
 '102614',
 '102715',
 '102816',
 '103010',
 '103111',
 '103212',
 '103414',
 '103515',
 '103818',
 '104012',
 '104416',
 '104820',
 '105014',
 '105115',
 '105216',
 '105620',
 '105923',
 '106016',
 '106319',
 '106521',
 '106824',
 '107018',
 '107220',
 '107321',
 '107422',
 '107725',
 '108020',
 '108121',
 '108222',
 '108323',
 '108525',
 '108828',
 '109123',
 '109325',
 '109830',
 '110007',
 '110411',
 '110613',
 '111009',
 '111211',
 '111312',
 '111413',
 '111514',
 '111716',
 '112112',
 '112314',
 '112516',
 '112819',
 '112920',
 '113215',
 '113316',
 '113417',
 '113619',
 '113821',
 '113922',
 '114116',
 '114217',
 '114318',
 '114419',
 '114621',
 '114823',
 '114924',
 '115017',
 '115219',
 '115320',
 '115724',
 '115825',
 '116120',
 '116221',
 '116423',
 '116524',
 '116726',

Bin´s instructions
- find the schaefer400 dlabel nii file (which is 64k (vertices?), cortex also 64k*1200 time series
- find cortex[schaefer==i] for i in range(400), then average
- then you get (400, 1200) ts
- then do np.corrcoef(ts) to get the correlation matrix
- But of note, some file contains medial wall (64k points) but some file excludes medial wall (59k points)

# PICK BACK WHERE LEFT OFF: what does 91282 correspond to in list length 1200


https://www.humanconnectome.org/hcp-protocols

number of voxels in a brain...? mni template for example if that is used


In [10]:
img = nib.load(datadir_day2+'100206'+'/MNINonLinear/Results/rfMRI_REST2_LR/rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii')  # loads nifti file
data = img.get_fdata()  # getting data from the image (functional activation), in an array format, len = 1200 (timeseries), each of 1200 lists has len = 91282
print(len(data))
print(data)
data_parc = hcp.parcellate(data, hcp.mmp)
print("---------------------------")
print(data_parc)
data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices
print(data_sub)

# img_cortex = data[:,hcp.struct.cortex] # cortex vertices
# node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts

# for i in range(400):
#     a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
#     ts = [None] * data_sub.shape[0]

#     for n in range(data_sub.shape[0]):
#         ts[n] = a[n].mean()
#         node[i] = ts

# # cortex parcels finished, now concatenate cortex and subcortex
# data_cor = np.array(node)

# # if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
# #data_cor_sub = np.concatenate((data_cor,data_sub.T))  

# # correlation
# corr_matrix = np.corrcoef(data_cor)   # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
# corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


1200
[[12632.64355469  9151.20410156  8184.51855469 ...  7863.93554688
   8073.78710938  8858.66992188]
 [12630.31054688  9103.41601562  8025.26171875 ...  7655.77148438
   8259.93652344  9764.984375  ]
 [12646.06835938  9065.97753906  8066.11669922 ...  8562.16210938
   8688.14550781  9706.890625  ]
 ...
 [12661.46972656  9139.45605469  8179.19775391 ...  8677.23828125
   8232.92578125  9534.68066406]
 [12707.67871094  8982.50195312  8112.90917969 ...  8515.78808594
   8354.12109375  9280.12792969]
 [12600.02734375  9066.55761719  8035.33789062 ...  8672.59667969
   8465.01757812  8685.29589844]]
-----------------------___
[[11063.20142042  9483.27184557 11905.34361294 ... 13143.608849
  13113.61538579 14696.7764619 ]
 [11065.83186026  9438.64029056 11906.62327817 ... 13198.14058008
  13085.54865245 14674.63039458]
 [11109.62838271  9461.28068814 11866.7725209  ... 13233.07380022
  13255.12199167 14647.91491847]
 ...
 [10969.83983494  9383.86484643 11813.21381579 ... 13191.64752425
  

In [26]:
len(data[0])

91282

In [12]:
64000*64000

4096000000

## Here trying to compare data matrices coming out of numpy.corrcoef vs nilearn.connectome.ConnectivityMeasure(kind='correlation') 

In [24]:
correlation_measure = ConnectivityMeasure(kind='correlation')  # need to change cov_estimator (default is LedoitWolf, but I don't know what other options there are - not specified in https://nilearn.github.io/modules/generated/nilearn.connectome.ConnectivityMeasure.html....)

These results below seem completely off... doesn't match the way it used to (it was closer back when we detected the mismatch... here nilearn function gives all correlations close to 1...) - maybe doing something wrong in the code here

In [25]:
img = nib.load(datadir_day2+'100206'+'/MNINonLinear/Results/rfMRI_REST2_LR/rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii')  # loads nifti file
data = img.get_fdata()  # getting data from the image (functional activation), in an array format, len = 1200 (timeseries), each of 1200 lists has len = 91282
data_parc = hcp.parcellate(data, hcp.mmp)
data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices


img_cortex = data[:,hcp.struct.cortex] # cortex vertices
node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts

for i in range(400):
    a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
    ts = [None] * data_sub.shape[0]

    for n in range(data_sub.shape[0]):
        ts[n] = a[n].mean()
        node[i] = ts

# cortex parcels finished, now concatenate cortex and subcortex
data_cor = np.array(node)

# if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
#data_cor_sub = np.concatenate((data_cor,data_sub.T))  

# correlation
corr_matrix = np.corrcoef(data_cor)   # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
#corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1
print(corr_matrix)

corr_matrix_nilearn = correlation_measure.fit_transform([data_cor])[0]
print(corr_matrix_nilearn)

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


[[1.         0.39772735 0.57170271 ... 0.18883729 0.22619851 0.34162415]
 [0.39772735 1.         0.44798775 ... 0.07143326 0.13434648 0.28310208]
 [0.57170271 0.44798775 1.         ... 0.2562463  0.20524264 0.44993065]
 ...
 [0.18883729 0.07143326 0.2562463  ... 1.         0.25283448 0.31006649]
 [0.22619851 0.13434648 0.20524264 ... 0.25283448 1.         0.40298602]
 [0.34162415 0.28310208 0.44993065 ... 0.31006649 0.40298602 1.        ]]
[[1.         0.99284147 0.99264198 ... 0.99247065 0.99233757 0.99230022]
 [0.99284147 1.         0.99279703 ... 0.99227433 0.99238478 0.99251997]
 [0.99264198 0.99279703 1.         ... 0.99217852 0.9922412  0.99223865]
 ...
 [0.99247065 0.99227433 0.99217852 ... 1.         0.99285836 0.99275786]
 [0.99233757 0.99238478 0.9922412  ... 0.99285836 1.         0.99287777]
 [0.99230022 0.99251997 0.99223865 ... 0.99275786 0.99287777 1.        ]]


Original code